# CNN for Image Classification

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from PIL import Image

## Import Data

In [7]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [9]:
batch_size = 20
trainloader = data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

batch_size = 4
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [10]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [14]:
dataiter = iter(trainloader)
images, labels = next(dataiter)

im = Image.fromarray((torch.cat(images.split(1,0),3).squeeze()/2*255+.5*255).permute(1,2,0).numpy().astype('uint8'))
im.save("train_pt_images.jpg")

print('train_pt_images.jpg saved.')
print('Ground truth labels:' + ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

train_pt_images.jpg saved.
Ground truth labels:  car   dog truck  ship  deer truck  deer   car  bird   cat   car horse plane  bird  bird   car horse   dog   cat  deer


## Import Test Set

## Network

In [17]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Training

In [21]:
def train(net, trainloader, num_epochs=2, learning_rate=0.001):
    
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    
    running_loss = 0.0
    
    for epoch in range(num_epochs):
        
        count = 0
        for X, y in trainloader:
            optimizer.zero_grad()
            l = loss(net(X), y)
            l.backward()
            optimizer.step()
            running_loss += l.item()
            count += 1
        
        print(f'Epoch: ', epoch, ' Training Loss: ', running_loss/count)
        running_loss = 0.0
        count = 0
        
    print('Training done.')
    
    torch.save(net.state_dict(), 'saved_model.pt')
    print('Model saved.')

In [22]:
net = Net()
train(net, trainloader)

Epoch:  0  Training Loss:  2.0379302422046663
Epoch:  1  Training Loss:  1.598342891049385
Training done.
Model saved.


## Test

In [28]:
def test(net, testloader):
    
    dataiter = iter(testloader)
    
    images, labels = next(dataiter)
    print('Ground-truth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
    
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(4)))

In [25]:
model = Net()
model.load_state_dict(torch.load('saved_model.pt'))

<All keys matched successfully>

In [29]:
test(model, testloader)

Ground-truth:    cat  ship  ship plane
Predicted:    cat  ship  ship  ship
